In [1]:
import z3
import json
import pprint
import numpy as np
import pandas as pd
#pd.set_option("display.max_rows", -1)
import random
import sys

# Compare alternative to networkx
#https://plot.ly/python/igraph-networkx-comparison/
import networkx as nx
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from ipywidgets import interact, interactive, widgets

import plotly.plotly as py
#username = 'eiselesr'
#api_key = 'Qf97n8ORXBbEpZEvD6HA'
#py.tools.set_credentials_file(username=username, api_key=api_key)


#DESCRIBEST HOW TO SET UP A HOSTED JUPYTER NOTEBOOK
#https://plot.ly/python/ipython-notebook-tutorial/


In [14]:
class a2n_solver(object):
    def __init__(self, raw):
        self.data = self.loadRaw(raw)
        self.resourceTypes = set([j for i in self.data for j in self.data[i]['resources'] ])
        #--Lists--
        self.nodeList = list()
        self.appList = list()
        self.nlappList = list() # List of apps that are dependencies
        self.prAppList = list() # List of primary apps that have dependencies
        self.init_lists() # provides values for previous 4 lists
        #--Matrices--
        self.a2a = self.init_a2a()
        self.a2a_df = pd.DataFrame(self.a2a, index=self.appList, columns=self.appList) 
        self.n2n = self.init_n2n()
        self.n2n_df = pd.DataFrame(self.n2n, index=self.nodeList, columns=self.nodeList) 
        # resources per node matrix
        self.rpn= [[self.data[j]['resources'][k] for j in self.nodeList] for k in self.resourceTypes]
        self.rpn_df = pd.DataFrame(self.rpn, index=self.resourceTypes, columns=self.nodeList)  
        # resources per app matrix
        self.rpa= [[self.data[i]['resources'][k] for i in self.appList] for k in self.resourceTypes]
        self.rpa_df = pd.DataFrame(self.rpa, index=self.resourceTypes, columns=self.appList)    
        # Placement matrix
        self.a2n = [ [ z3.Real("a%s%s" % (i+1, j+1)) for j in range(len(self.nodeList)) ]
                                                     for i in range(len(self.appList))]
        self.a2n_df = pd.DataFrame(self.a2n, index=self.appList, columns=self.nodeList)  
        self.initial_deployment() # this must happen before the cost_obj, otherwise the variables are used.
        #--------- CONSTRAINTS------------------
        self.a2n_domain = [z3.Or(self.a2n_df.loc[app, node]==0, self.a2n_df.loc[app, node]==1) \
                      for app in self.nlappList \
                      for node in self.nodeList]
        # Max capacity constraint
        # Resources required by apps deployed on a node cannot exceed that nodes resources
        self.app_util = self.rpa_df.dot(self.a2n_df)
        self.node_util = self.rpn_df - self.app_util
        self.rsrc_constraint = [self.node_util.loc[rsrc,node]>=0 \
                                       for rsrc in self.resourceTypes \
                                       for node in self.nodeList]
        # Must Deploy Dependencies constraint
        # Assuming the primary app is deployed, all of it's dependencies must be deployed
        self.mdd = [self.a2n_df.loc[dep].sum() > 0 for dep in self.nlappList]
        #--------- OPTIMIZATION OBJECTIVES------------------
        
        self.lat_obj = self.genLatencyObj(self.a2n_df)        
        self.cost_obj = self.genCostObj(self.a2n_df)
        self.rsrc_util = None
        self.util_obj = self.utilObj(self.a2n_df)
        self.balance_obj = self.balanceObjective(self.a2n_df)
        
        
        
        self.scale_latency = None
        self.scale_rsrc = None
        self.max_edge_latency = 10
        self.normalize()
        self.p = .5
        self.q = 0
        # -------- Solution ---------------------------
        self.a2n_df_r = None
        self.optimize(self.p, self.q)
         
        
    def loadRaw(self, raw):
        with open(raw) as file:
            data = json.load(file)
        return data
    
    def init_lists(self):        
        for i in self.data:
            if self.data[i]['type']=='node':
                self.nodeList.append(i)
            if self.data[i]['type']=='app':
                self.appList.append(i)
                if self.data[i]['dependencies']['nonlocal']:
                    self.prAppList.append(i)
                    for dep in self.data[i]['dependencies']['nonlocal']:
                        self.nlappList.append(dep)
        self.nodeList = sorted(self.nodeList)
        self.appList = sorted(self.appList)
        self.prAppList = sorted(self.prAppList)
        self.nlappList = sorted(self.nlappList)
                        
    def init_a2a(self):
        a2a = []
        for app in self.appList:
            nl_deps = self.data[app]['dependencies']['nonlocal']
            deps = []
            for dep in self.appList:
                if dep in nl_deps:
                    deps.append(1)
                else: 
                    deps.append(0)
            a2a.append(deps)
        return a2a
    
    def init_n2n(self):
        n2n = []
        for node in self.nodeList:
            id = self.data[node]['id']
            temp = []    
            for n2 in self.nodeList:
                id2 = self.data[n2]['id']
                lat = self.data[node]['latencies']['n%s%s' %(id, id2)]
                temp.append(lat)
            n2n.append(temp)  
        return n2n        
    
    def initial_deployment(self):
        self.a2n_df.loc['APP1'] = 1
        
    def genLatencyObj(self, a2n_df):
        numNodes = len(self.nodeList) # number of FSSNs
        markings = list() # The equations for each possible placement of the dependencies

        for srcNode in self.nodeList:
            #print (srcNode)
            # -- Check all nodes for all primary apps.--
            for prApp in self.prAppList:
                if a2n_df.loc[prApp, srcNode] == 1:   ### If a primary app is present 
                    ## ANOTHER FOR LOOP IF THERE ARE MULTIPLE DEPENDENCIES
                    primaryDeps = [dep for dep in self.appList if self.a2a_df.loc[prApp, dep]==1]
                    for dep in primaryDeps:
                        # iterate through the 2^numNodes potential dependency placements
                        # and save them as binary representation to look up when running the optimizer
                        for i in range(2**numNodes):
                            mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                            #print(mark)
                            # The dependency must be place somewhere. So this skips the case where i=0
                            # This may not be necessary thanks to the constraint. 
                            if mark == format(0,'b').zfill(numNodes):
                                continue

                            marking = list() # list for placment equations e.g (1 - a11)*(1 - a12)*a13
                            latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                            #print ("-----NEW MARKING-----")
                            # iterate through nodes using the binary placement to generate marking equation
                            for i, node in enumerate(self.nodeList):
                                #print ("mark %s" %mark)
                                #print ("i %s " %i)
                                #print ("mark[i] %s" %type(int(mark[i]))) 
                                # add marking to list and if binary has a 1, add latency to list of possible latencies
                                if int(mark[i]) == 1:
                                    marking.append(a2n_df.loc[dep, node]) #a_ij
                                    latency.append(self.n2n_df.loc[srcNode, node])
                                    #latency.append(random.randint(0,10))
                                if int(mark[i]) == 0:
                                    marking.append(1 - a2n_df.loc[dep,node]) # 1- a_ij
                            #print("latency %s" %latency)                    
                            #print("min latency %s" %min(latency).item())
                            #print("min latency %s" %min(latency))
                            #print("type(min(latency)) = %s" %type(min(latency)))
                            #print("marking: %s" %marking)
                            #print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                            #print("marking * latency: %s" %(z3.Product(marking) * min(latency)))
                            #markings.append(z3.Product(marking) * min(latency).item())
                            markings.append(z3.Product(marking) * min(latency))
            
        lat_obj = z3.Sum(markings)
        return lat_obj
    
    def genCostObj(self, a2n_df):
        costs = list()
        for app in self.appList:
            for rsrc in self.resourceTypes:                
                try:
                    cost = a2n_df.loc[app]*(self.rpa_df.loc[rsrc, app]).item()
                    #print("cost %s" %cost)
                    costs.append(sum(cost))
                except Exception as inst:
                    print("getCostObj Exception type: %s" %type(inst))
                    print (self.rpa_df.loc[:, app])
        total_cost = sum(costs)
        #print(type(total_cost))
        #print(total_cost.is_real())
        #print(total_cost.is_int())
        return total_cost
    
    def getNodeDemand(self, node, a2n_df):
        ''' returns a dictionary of the resource consumption on the specified node'''
        node_demand = {}
        for rsrc in self.resourceTypes:
            node_demand[rsrc] = list()
            temp = list()
            for app in self.appList:
                temp.append(a2n_df.loc[app, node] * (self.rpa_df.loc[rsrc, app]).item())
            node_demand[rsrc] = z3.Sum(temp)
        return node_demand
    
    def getNodeSupply(self, node):
        node_supply={}
        for rsrc in self.resourceTypes:
            node_supply[rsrc] = self.rpn_df.loc[rsrc, node]
        return node_supply
    
    def getNetworkDemand(self, a2n_df):
        network_demand = {}
        for rsrc in self.resourceTypes:
            network_demand[rsrc] = list()
            temp = list()
            for node in self.nodeList:
                temp.append(self.getNodeDemand(node, a2n_df)[rsrc])
            network_demand[rsrc] = z3.Sum(temp)
        return network_demand
    
    def getNetworkSupply(self):
        network_supply = {}
        for rsrc in self.resourceTypes:
            network_supply[rsrc] = 0
            for node in self.nodeList:
                network_supply[rsrc] = network_supply[rsrc] + self.getNodeSupply(node)[rsrc]
        return network_supply
    
    def getNodeUtil(self, node, a2n_df):
        util = {}
        for rsrc in self.resourceTypes:
            demand = self.getNodeDemand(node, a2n_df)[rsrc]
            supply = self.getNodeSupply(node)[rsrc]
            util[rsrc] = demand/supply
        return util
                
    def getNetworkUtil(self, a2n_df):
        util = {}
        for rsrc in self.resourceTypes:
            demand = self.getNetworkDemand(a2n_df)[rsrc]
            supply = self.getNetworkSupply()[rsrc]
            util[rsrc] = demand/supply
        return util
    
    def balanceObjective(self, a2n_df):
        netUtil = self.getNetworkUtil(a2n_df)
        #print("netUtil: %s" % netUtil)
        sum_sq_err = {}
        for rsrc in self.resourceTypes:
            sum_sq_err[rsrc] = list()
            temp = list()
            for node in self.nodeList:
                nodeUtil = self.getNodeUtil(node, a2n_df)
                #print("nodeUtil: %s" % nodeUtil)
                sqdiff = (nodeUtil[rsrc] - netUtil[rsrc])**2
                #print("sqdiff: %s" %z3.simplify(sqdiff)) 
                temp.append(sqdiff)
            sum_sq_err[rsrc] = z3.Sum(temp)
        return sum_sq_err         
            
    
    def utilObj(self, a2n_df):
        demand = {}
        supply = {}
        util = {}
        totalDemand = 0
        totalSupply = 0
        totalUtil = 0
        for rsrc in self.resourceTypes:
            demand[rsrc] = list()
            supply[rsrc] = 0
            util[rsrc] = 0
            
            nodeDemand = 0
            for node in self.nodeList:
                appsDemand = list()
                for app in self.appList:
                    appDemand = a2n_df.loc[app, node] * (self.rpa_df.loc[rsrc, app]).item() 
                    #print("appDemand: %s" %appDemand)
                    appsDemand.append(appDemand)                
                nodeDemand = z3.simplify(z3.Sum(appsDemand))
                #print("nodeDemand: %s" %nodeDemand)
                demand[rsrc].append(z3.simplify(z3.Sum(nodeDemand)))
                supply[rsrc] = supply[rsrc] + self.rpn_df.loc[rsrc, node] #cumulate node resources
             
            #print("num: %s" %z3.simplify(z3.Sum(demand[rsrc])))
            #print("den: %s" %supply[rsrc])
            util[rsrc] = z3.simplify(z3.Sum(demand[rsrc])/ supply[rsrc])
            totalDemand = totalDemand + z3.Sum(demand[rsrc])
            totalSupply = totalSupply + supply[rsrc]
            
        totalUtil = z3.simplify(totalDemand/ totalSupply)
        #print("demand: %s" %demand)
        #print("supply: %s" %supply)
        #print("totaldemand: %s" %z3.simplify(totalDemand))
        #print("totalSuppply: %s" %totalSupply)
        #print("util: %s" %util)
        #print("totalUtil : %s " %totalUtil)    
        self.rsrc_util = util
        return util # retun utilization of individual resources
        #return totalUtil # return utilization as if resources were all the same.             
        
   
    def normalize(self):
        min_latency = 0 #app is deployed everywhere
        #max_latency = max([sum(self.n2n_df.loc[node, :]) for node in self.nodeList]) #app is deployed on node with highest total latency
        max_latency = self.max_edge_latency * (len(self.nodeList)-1) * (len(self.nlappList))
        self.scale_latency = 1/(max_latency - min_latency)

        #min_rsrc = in nlappList
        # every app deployed on every node [(sum of apps costs) * num of nodes]
        max_rsrc = max([sum(self.rpa_df.loc[rsrc, :]) for rsrc in self.resourceTypes])*len(self.nodeList)
        #print("max_rsrc: %s" %max_rsrc)
        min_rsrc = 0 #self.rpa_df.loc['memory', 'APP2' ] + 5*self.rpa_df.loc['memory', 'APP1']
        self.scale_rsrc = 1/(max_rsrc - min_rsrc)
          
    def optimize(self, p, q):
        self.p = p
        self.q = q
        opt = z3.Optimize()
        constraints = self.a2n_domain + self.rsrc_constraint + self.mdd
        #print("constraints")
        #display(constraints)
        #opt.add(constraints)
        opt.assert_exprs(constraints)
        #print("test")
        #pprint.pprint(type(self.cost_obj))
        #pprint.pprint(self.cost_obj)
         
        # minimize latency and cost
        #opt.minimize(p*self.lat_obj*self.scale_latency + (1-p)*self.cost_obj*self.scale_rsrc)
        
        #minimize latency and total resource utilization
        #opt.minimize(p*self.lat_obj*self.scale_latency + (1-p)*self.util_obj)
        
        #minimize latency and resource imbalance
        #opt.minimize(p*self.lat_obj*self.scale_latency + (1-p)*self.balance_obj['ram'])
        
        #minimize imbalance, latency, and utilization
        opt.minimize(q*self.balance_obj['ram'] + (1-q)*(p*self.lat_obj*self.scale_latency + (1-p)*self.util_obj['ram']))
        
        opt.check()

        opt_model = opt.model()
        #print("optimization model \n %s \n" %opt_model)

        a2n_df_r = self.a2n_df.copy()
        for app in self.nlappList:
            #print('App %s' %app)
            for node in self.nodeList:
                #print('Node %s' %node)
                a2n_df_r.loc[app, node] = opt_model.eval(self.a2n_df.loc[app, node])
        self.a2n_df_r = a2n_df_r
        #return a2n_df_r        
    
    def updateLatency(self, edge, latency):
        print("edge: %s type: %s" %(edge, type(edge)))
        print("before lat update %s" %self.n2n_df.loc[tuple(edge)])
        self.n2n_df.loc[tuple(edge)] = latency
        self.n2n_df.loc[edge[1], edge[0]] = latency
        print("after lat update %s" %self.n2n_df.loc[tuple(edge)])
        #print("edge in n2n %" %self.n2n_df.loc[edge])
        #for i, edge in enumerate(edges):
        #    print(i)
        #    print("update latency %s" %edge)
        #    self.n2n_df.loc[edge] = latency[i]
        #    self.n2n_df.loc[edge[1], edge[0]] = latency[i]
        self.lat_obj = self.genLatencyObj(self.a2n_df)
        #self.normalize()
        self.optimize(self.p, self.q)
        return self.a2n_df_r
    
    def printCostFunction(self):
        
        
        rsrc_util = self.utilObj(self.a2n_df_r)['ram']
        print("system resource utilization %s" %rsrc_util)
        
        print("Cost Obj")
        #pprint.pprint(self.genCostObj(self.a2n_df_r), compact=True, width=120)
        rsrc_cost = z3.simplify(self.genCostObj(self.a2n_df_r))
        latency_cost = z3.simplify(self.genLatencyObj(self.a2n_df_r))
        balance_cost = z3.simplify(self.balanceObjective(self.a2n_df_r)['ram'])
        print("rsrc cost: %s latency cost:%s util:%s imbalance:%s" %(rsrc_cost, latency_cost, rsrc_util, balance_cost))
        
        # Objective latency & cost
        #print("p*latency*scale_latency + (1-p)*resource*scale_rsrc")
        #result = self.p*latency_cost*self.scale_latency + (1-self.p)*rsrc_cost*self.scale_rsrc
        #display(result)
        #print("simplify: lat=%s + rsrc=%s" %(z3.simplify(self.p*latency_cost*self.scale_latency).as_decimal(5),
        #                                     z3.simplify((1-self.p)*rsrc_cost*self.scale_rsrc).as_decimal(5)))
        
        #Objective latency & utilization
        #print("p*latency*scale_latency + (1-p)*util")
        #result = self.p*latency_cost*self.scale_latency + (1-self.p)*rsrc_util
        #display(result)
        #print("simplify: lat=%s + util=%s" %(z3.simplify(self.p*latency_cost*self.scale_latency).as_decimal(5),
        #                                     z3.simplify((1-self.p)*rsrc_util).as_decimal(5)))
        
        
        
        #Objective latency, imbalance
        #print("p*latency*scale_latency + (1-p)*balance")
        #result = self.p*latency_cost*self.scale_latency + (1-self.p)*balance_cost
        #display(result)
        #print("simplify: lat=%s + imbalance=%s" %(z3.simplify(self.p*latency_cost*self.scale_latency).as_decimal(5),
        #                                     z3.simplify((1-self.p)*balance_cost).as_decimal(5)))
        
        #Objective imbalance, latency & utilization
        print("q*imbalance + (1-q)*p*latency*scale_latency + (1-q)(1-p)*utilization")
        result = self.q*balance_cost + (1-self.q)*(self.p*latency_cost*self.scale_latency + (1-self.p)*rsrc_util)       
        display(result)
        #print("simplify: imbalance=%s  lat=%s util=%s" 
        #      %(z3.simplify(self.q * balance_cost).as_decimal(5),
        #        z3.simplify((1-self.q)*self.p*latency_cost*self.scale_latency).as_decimal(5),
        #        z3.simplify((1-self.q)*(1-self.p)*balance_cost).as_decimal(5)))
        
        print("Result %s" %z3.simplify(result).as_decimal(5))
        
    def printDebug(self):
        print("latency objective. Should be symbolic. ")
        pprint.pprint(z3.simplify(s.lat_obj)) 
        
    
s = a2n_solver('simple-2rsrc.json')
display(s.a2n_df_r)

#display(s.nodeList, s.appList, s.nlappList, s.prAppList, s.a2a_df, s.n2n_df, s.rpn_df, s.rpa_df, s.a2n_df)
#display(s.a2n_domain)
#display(s.a2n_df, s.rsrc_constraint, s.mdd, s.cost_obj)
#display(s.lat_obj)
#display(s.scale_latency, s.scale_rsrc)
#display(s.a2n_df_r)
#new_a2n_df_r = s.updateLatency([('FSSN1', 'FSSN2'), ('FSSN1', 'FSSN5'), ('FSSN2', 'FSSN5')], [10, 11, 12])
#new_a2n_df_r = s.updateLatency(['FSSN1', 'FSSN2'], 10)
#display(new_a2n_df_r)
#display(s.n2n_df)
#display(z3.simplify(s.genCostObj(new_a2n_df_r)))
#display(s.genLatencyObj(new_a2n_df_r))
#display(z3.simplify(s.genLatencyObj(new_a2n_df_r)))
#s.printCostFunction()
#display(s.getNodeDemand('FSSN1', s.a2n_df))
#display(s.getNodeSupply('FSSN1'))
#display(s.getNetworkDemand(s.a2n_df))
#display(s.getNetworkSupply())
#display(s.getNodeUtil('FSSN1', s.a2n_df))
#display(s.getNetworkUtil(s.a2n_df))
#display(s.balanceObjective(s.a2n_df))

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,1,1,1
APP3,0,1,1,1,1


In [30]:
display(s.a2n_df)
print(s.nlappList)
s.a2n_df.loc[s.nlappList].values

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,a21,a22,a23,a24,a25
APP3,a31,a32,a33,a34,a35


['APP2', 'APP3']


array([[a21, a22, a23, a24, a25],
       [a31, a32, a33, a34, a35]], dtype=object)

In [3]:
s.util_obj

{'ram': 1/20*a21 +
 1/20*a31 +
 1/20*a22 +
 1/20*a32 +
 1/20*a23 +
 1/20*a33 +
 1/20*a24 +
 1/20*a34 +
 1/20*a25 +
 1/20*a35, 'storage': 1/48*a21 +
 1/48*a31 +
 1/48*a22 +
 1/48*a32 +
 1/48*a23 +
 1/48*a33 +
 1/48*a24 +
 1/48*a34 +
 1/48*a25 +
 1/48*a35}

In [4]:
print("ram imbalance = %s" %z3.simplify(s.balanceObjective(s.a2n_df_r)['ram']))
print("storage imbalance = %s " %z3.simplify(s.balanceObjective(s.a2n_df_r)['storage']))
s.printCostFunction()

ram imbalance = 1/5
storage imbalance = 29/576 
system resource utilization 2/5
Cost Obj
rsrc cost: 16384 latency cost:6 util:2/5 imbalance:1/5
q*imbalance + (1-q)*p*latency*scale_latency + (1-q)(1-p)*utilization


0*1/5 + 1*(1/2*6*1/80 + 1/2*2/5)

Result 0.2375


In [5]:
a = (1 ,2 )
type(a)
b = [1, 2]
type(b)
tuple(b)

(1, 2)

In [6]:
s.n2n_df.loc[tuple(['FSSN1', 'FSSN2'])]


3

In [7]:
s = a2n_solver('simple-2rsrc.json')
def test(p, q):
    print(p)
    s.optimize(p, q)     
    display(s.a2n_df_r)
    print("scale resource: %s \n scale latency: %s" %(s.scale_rsrc, s.scale_latency))

    s.printCostFunction()
    

p = widgets.FloatSlider(min=0, max=1, step=0.1, value=.6, continuous_update=False)
q = widgets.FloatSlider(min=0, max=1, step=0.1, value=.6, continuous_update=False)

interactive_plot = interactive(test, p=p, q=q)
interactive_plot

constraints


[Or(a21 == 0, a21 == 1),
 Or(a22 == 0, a22 == 1),
 Or(a23 == 0, a23 == 1),
 Or(a24 == 0, a24 == 1),
 Or(a25 == 0, a25 == 1),
 Or(a31 == 0, a31 == 1),
 Or(a32 == 0, a32 == 1),
 Or(a33 == 0, a33 == 1),
 Or(a34 == 0, a34 == 1),
 Or(a35 == 0, a35 == 1),
 8192 - (0 + 1024*a21 + 1024*a31) >= 0,
 8192 - (0 + 1024*a22 + 1024*a32) >= 0,
 16384 - (0 + 1024*a23 + 1024*a33) >= 0,
 8192 - (0 + 1024*a24 + 1024*a34) >= 0,
 8192 - (0 + 1024*a25 + 1024*a35) >= 0,
 4096 - (0 + 1024*a21 + 1024*a31) >= 0,
 4096 - (0 + 1024*a22 + 1024*a32) >= 0,
 4096 - (0 + 1024*a23 + 1024*a33) >= 0,
 4096 - (0 + 1024*a24 + 1024*a34) >= 0,
 4096 - (0 + 1024*a25 + 1024*a35) >= 0,
 a21 + a22 + a23 + a24 + a25 > 0,
 a31 + a32 + a33 + a34 + a35 > 0]

In [9]:
class visualizeGraph():
    def __init__(self, s):
        self.G = nx.Graph()
        self.G1 = self.placeStations(self.G)
        self.edge_annotations = self.annotateEdges(self.G1)
        self.G2 = self.placeApps(self.G1)
        self.node_trace = self.nodeTrace(self.G2)
        self.edge_traces = self.edgeTraces(self.G1)
        
    def placeStations(self, G):    
        for node1 in s.a2n_df_r.keys():
            #print("node1 is %s" %node1)
            G.add_node(node1, attr_dict=dict(s.rpn_df.loc[:, node1]))
            #print("G.nodes: %s" %G.nodes())
            id1 = s.data[node1]['id']
            for node2 in s.nodeList:
                id2 = s.data[node2]['id']
                if id1 != id2:            
                    G.add_edge(node1, node2, latency=s.n2n_df.loc[node1,node2])  
        #circular_layout(G, dim=2, scale=1, center=None)
        pos = nx.circular_layout(G)
        #https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.classes.function.set_node_attributes.html
        nx.set_node_attributes(G, 'pos', pos)
        nx.set_node_attributes(G, 'color', 65)
        #self.G1 = G
        return G
    
    def annotateEdges(self, G):    
        # The normal edge annotations are on one of the edge endpoints
        # This is to place an annotation on the middle of the each edge
        edge_annotations = go.Annotations()
        for edge in G.edges():
            x0, y0 = G.node[edge[0]]['pos']
            x1, y1 = G.node[edge[1]]['pos']
            mx, my = [(x1+x0)*.5,(y1+y0)*.5]
            edge_annotations.append(go.Annotation(
                x = mx,
                y = my, 
                xref='x',
                yref='y',
                text = s.n2n_df.loc[edge]))
            #print("edge: %s lat: %s" %(edge, n2n_df.loc[edge]))
            #print("mx: %s, my: %s" %(mx, my))
            #print("mx: %s, my: %s" %(mx, my))  
        #self.edge_annotations = edge_annotations
        return(edge_annotations)
    
    def placeApps(self, G):
        appColor = {}
        for node in s.nodeList:
            #print("\n before app Graph Edges:")
            #pprint.pprint(G.edges())
            appG = nx.Graph()
            numApps = 1 #used to increment color when there is a new app

            for app in s.appList:
                #---- Check type of a2n matrix entries. If they are a z3 type convert to an int.----
                if type(s.a2n_df_r.loc[app, node]) ==z3.RatNumRef:
                    val = (s.a2n_df_r.loc[app, node]).as_fraction()
                else:
                    val = s.a2n_df_r.loc[app, node]
                #---COLOR APPS UNIQUELY---
                if val == 1:
                    if app in appColor:
                        color = appColor[app]
                    else:
                        color = 65+10*numApps
                        appColor[app] = color
                    #---Add appnodes their edges to the station node---
                    appG.add_node(app+node, attr_dict=dict(s.rpa_df.loc[:, app]), color=color)
                    appG.add_edge(node, app+node) #adding the edge also adds the node
                numApps += 1
            #---Place apps---            
            app_pos = nx.circular_layout(appG, scale=.05, center=G.node[node]['pos'])
            nx.set_node_attributes(appG, 'pos', app_pos)
            #---combine app graph with station graph            
            G = nx.compose(appG, G) # Attributes from H take precedent over attributes from G.             
            #print("\n node %s" %node)
            #pprint.pprint(G.nodes(data=True), width=120)
        #self.G2 = G
        return G    
    
    def nodeTrace(self, G):
        node_trace = go.Scatter(
                x=[],
                y=[],
                text=[],
                mode='markers',
                hoverinfo='text',
                textposition='bottom',
                marker=go.Marker(
                    showscale=True,
                    # colorscale options
                    # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                    # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                    colorscale='RdBu',
                    reversescale=True,
                    color=[],
                    size=[],
                    #colorbar=dict(
                    #    thickness=15,
                    #    title='Node Connections',
                    #    xanchor='left',
                    #    titleside='right'
                    #),
                    #line={'width':10}
                )
            )

        #thickness = 1
        for node in reversed(sorted(G.nodes())):
            #print(node)
            #print(G.node[node]['pos'])
            x, y = G.node[node]['pos']
            node_trace['x'].append(x)
            node_trace['y'].append(y)
            degree = nx.degree(G, node)
            #node_info = node + ": " + str(G.node[node]['memory'])
            #node_trace['text'].append(node_info)
            node_trace['marker']['size'].append(15*degree)
            node_trace['marker']['color'].append(G.node[node]['color'])
            #node_trace['marker']['line'].append(thickness)
            #thickness = thickness + 1
        #self.node_trace = node_trace
        return node_trace  
    
    def edgeTrace(self, width):
        edge_trace = go.Scatter(
            x=[],
            y=[],
            line=go.Line(width=width,color='#888'),
            hoverinfo='text',
            mode='lines',
            text=[],
            #textposition='middleright'
            )
        return edge_trace  
    
    def edgeTraces(self, G):
        edge_traces = []
        #---Build edge traces to put in figure.--- 
        for edge in G.edges():
            #print(edge)
            if edge[0] in s.n2n_df and edge[1] in s.n2n_df:
                #print("%s in n2n" %str(edge))
                latency = s.n2n_df.loc[edge]
                edge_trace = self.edgeTrace(latency)
            else:
                edge_trace = edgeTrace(0)                

            x0, y0 = G.node[edge[0]]['pos']
            x1, y1 = G.node[edge[1]]['pos']
            edge_info = edge
            edge_trace['text'].append(edge_info)
            edge_trace['x'] += [x0, x1]
            edge_trace['y'] += [y0, y1]
            edge_traces.append(edge_trace)
        #self.edge_traces = edge_traces
        return edge_traces    

    def plot(self):
        
        self.edge_annotations = self.annotateEdges(self.G1)
        self.G2 = self.placeApps(self.G1)
        self.node_trace = self.nodeTrace(self.G2)
        self.edge_traces = self.edgeTraces(self.G1)
        
        traces = self.edge_traces.copy() # For some reason this needs to be an explicit copy. Otherwise when I do traces.append it appends to self.edge_traces
        traces.append(self.node_trace) 
        #pprint.pprint(traces)
        
        fig = go.Figure(data=go.Data(traces),
                     layout=go.Layout(
                         title='<br>App placement graph',
                         titlefont=dict(size=16),
                         showlegend=False,
                         hovermode='closest',
                         margin=dict(b=20,l=5,r=5,t=40),
                         #annotations=[ dict(
                         #    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                         #    showarrow=False,
                         #    xref="paper", yref="paper",
                         #    x=0.005, y=-0.002 ) ],
                         annotations=self.edge_annotations,
                         xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                         yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False),
                         #sliders = sliders
                     ))
        return py.iplot(fig, filename='networkx')    
   
        
vg = visualizeGraph(s)  
#vg.G1.nodes(data=True)
#display(vg.G1.edges(data=True))
#vg.edgeAnnotations
#vg.G2.nodes(data=True)
#vg.node_trace
#vg.edge_traces

display(vg.plot())

In [10]:
display(s.a2n_df_r)
display(s.n2n_df)
display(s.p)

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,1,1,1,1
APP3,1,1,1,1,1


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
FSSN1,0,3,4,5,6
FSSN2,3,0,5,6,7
FSSN3,4,5,0,7,8
FSSN4,5,6,7,0,9
FSSN5,6,7,8,9,0


0.6

In [12]:
import numpy as np
from IPython.display import clear_output

#https://stackoverflow.com/questions/37622023/how-to-sort-key-names-in-ipywidgets-interactive-widgets-slider
s = a2n_solver('simple-2rsrc.json')
vg = visualizeGraph(s)

def myfct(change,list_widgets):
    clear_output()
    #display(s.a2n_df)
    
    if (change.owner.description == 'p'):
        #print("p value: %s"  %change.new)
        s.optimize(p=change.new, q=s.q)      
    elif(change.owner.description == 'q'):
        #print("q value: %s"  %change.new)
        s.optimize(p=s.p, q=change.new)           
    else:
        edge = json.loads(change.owner.description)
        value = change.new
        s.updateLatency(edge, value)
    s.printCostFunction()
       
    display(s.n2n_df)
    display(s.a2n_df_r)
        
    display(vg.plot())
    
        

# -------- SLIDER WIDGETS -----------
list_widgets=[]
list_widgets.append(widgets.FloatSlider(min=0, max=1, step=0.1, value=s.p, continuous_update=False, description='p'))
list_widgets.append(widgets.FloatSlider(min=0, max=1, step=0.1, value=s.q, continuous_update=False, description='q'))

for edge in sorted(vg.G1.edges()):
    #lat = vg.G1.edge[edge[0]][edge[1]]['latency']
    lat = s.n2n_df.loc[edge]
    list_widgets.append(widgets.IntSlider(min=0, max=s.max_edge_latency, step=1, value=lat, continuous_update=False, description= json.dumps(edge)))

for widget in list_widgets:
    widget.observe(lambda change:myfct(change,list_widgets),names='value',type='change')
     

w_left = []
w_middle=[]
w_right =[]

for i, widget in enumerate(list_widgets):
    if i%3==0:
        w_left.append(widget)
    if i%3==1:
        w_middle.append(widget)
    if i%3==2:
        w_right.append(widget)

left = widgets.VBox(w_left)
middle = widgets.VBox(w_middle)
right = widgets.VBox(w_right)
        

w = widgets.HBox([left, middle, right])


# display the widgets
display(w)        

constraints


[Or(a21 == 0, a21 == 1),
 Or(a22 == 0, a22 == 1),
 Or(a23 == 0, a23 == 1),
 Or(a24 == 0, a24 == 1),
 Or(a25 == 0, a25 == 1),
 Or(a31 == 0, a31 == 1),
 Or(a32 == 0, a32 == 1),
 Or(a33 == 0, a33 == 1),
 Or(a34 == 0, a34 == 1),
 Or(a35 == 0, a35 == 1),
 8192 - (0 + 1024*a21 + 1024*a31) >= 0,
 8192 - (0 + 1024*a22 + 1024*a32) >= 0,
 16384 - (0 + 1024*a23 + 1024*a33) >= 0,
 8192 - (0 + 1024*a24 + 1024*a34) >= 0,
 8192 - (0 + 1024*a25 + 1024*a35) >= 0,
 4096 - (0 + 1024*a21 + 1024*a31) >= 0,
 4096 - (0 + 1024*a22 + 1024*a32) >= 0,
 4096 - (0 + 1024*a23 + 1024*a33) >= 0,
 4096 - (0 + 1024*a24 + 1024*a34) >= 0,
 4096 - (0 + 1024*a25 + 1024*a35) >= 0,
 a21 + a22 + a23 + a24 + a25 > 0,
 a31 + a32 + a33 + a34 + a35 > 0]

system resource utilization 1/10
Cost Obj
rsrc cost: 4096 latency cost:36 util:1/10 imbalance:1/5
q*imbalance + (1-q)*p*latency*scale_latency + (1-q)(1-p)*utilization


0*1/5 + 1*(1/5*36*1/80 + 4/5*1/10)

Result 0.17


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
FSSN1,0,3,4,5,6
FSSN2,3,0,5,6,7
FSSN3,4,5,0,7,8
FSSN4,5,6,7,0,9
FSSN5,6,7,8,9,0


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,0,0
APP3,1,0,0,0,0


In [13]:
display(s.a2n_df_r)
s.printCostFunction()
print("p: %s q: %s" %(s.p, s.q))
s.a2n_df_r.loc['APP3', 'FSSN1'] = 0
s.a2n_df_r.loc['APP2', 'FSSN2'] = 0


s.a2n_df_r.loc['APP3', 'FSSN3'] = 0 
s.a2n_df_r.loc['APP2', 'FSSN4'] = 0
s.a2n_df_r.loc['APP3', 'FSSN5'] = 0
s.printCostFunction()
s.a2n_df_r

,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,0,0
APP3,1,0,0,0,0


system resource utilization 1/10
Cost Obj
rsrc cost: 4096 latency cost:36 util:1/10 imbalance:1/5
q*imbalance + (1-q)*p*latency*scale_latency + (1-q)(1-p)*utilization


0*1/5 + 1*(1/5*36*1/80 + 4/5*1/10)

Result 0.17
p: 0.2 q: 0
system resource utilization 1/20
Cost Obj
rsrc cost: 2048 latency cost:18 util:1/20 imbalance:1/20
q*imbalance + (1-q)*p*latency*scale_latency + (1-q)(1-p)*utilization


0*1/20 + 1*(1/5*18*1/80 + 4/5*1/20)

Result 0.085


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,0,0
APP3,0,0,0,0,0


In [ ]:
import operator

def bruteForce(numVars):
    my_list = list()
    my_marking = list()
    for i in range(2**numVars):
        
        marking = format(i, 'b').zfill(numVars) # assign values to variables
        #my_a21 = int(marking[0])
        #print(small_obj(a21=my_a21))
        if marking == format(0,'b').zfill(numVars):
            continue
        my_cost = checkCost(q,a21=z3.RealSort().cast(marking[0]), a31=z3.RealSort().cast(marking[1]), 
                           a22=z3.RealSort().cast(marking[2]), a32=z3.RealSort().cast(marking[3]), 
                           a23=z3.RealSort().cast(marking[4]), a33=z3.RealSort().cast(marking[5]), 
                           a24=z3.RealSort().cast(marking[6]), a34=z3.RealSort().cast(marking[7]), 
                           a25=z3.RealSort().cast(marking[8]), a35=z3.RealSort().cast(marking[9]))
        my_cost_py = z3.simplify(my_cost).as_decimal(5)
        my_list.append(my_cost_py)
        my_marking.append(marking)
        
        #if (i < 20) or (i>1000):
        #    print(my_cost_py, marking)
        #if (i>20):
         #   break
        #print(my_cost_py)
    #print(max(my_list), min(my_list))
    max_index, max_value = max(enumerate(my_list), key=operator.itemgetter(1))
    min_index, min_value = min(enumerate(my_list), key=operator.itemgetter(1))
    #return(my_marking[max_index], max_value, my_marking[min_index], min_value)
    
    marking = my_marking[min_index]
    my_cost = checkCost(q,a21=z3.RealSort().cast(marking[0]), a31=z3.RealSort().cast(marking[1]), 
                           a22=z3.RealSort().cast(marking[2]), a32=z3.RealSort().cast(marking[3]), 
                           a23=z3.RealSort().cast(marking[4]), a33=z3.RealSort().cast(marking[5]), 
                           a24=z3.RealSort().cast(marking[6]), a34=z3.RealSort().cast(marking[7]), 
                           a25=z3.RealSort().cast(marking[8]), a35=z3.RealSort().cast(marking[9]))
    
    return(my_marking[min_index], my_cost, z3.simplify(my_cost).as_decimal(5))